In [6]:
import pymongo

from lib.common import MONGODB_URI

client = pymongo.MongoClient(MONGODB_URI)
# define pipeline


아래 파이프라인은 여러개의 조건을 검색합니다. 

* filter : 해당 조건이 True 이어야 하나 Score 는 영향을 미치지 않음 
* must   : 해당 조건이 True 이어야 하고 Score 에도 영향을 미침 
* should : 매칭이 된다면 Score 가 올라갑니다.  

In [7]:


# 2013 년부터 2015년에 year 필드 레인지에서 
# title 에 snow 가 들어가는 document 를  찾아라 
# 매칭이 되면 Score 가 변하는 게 중요 해당 Score 가 constant 인것을 주목 

pipeline = [
  {'$search': {
      'compound': {
        'filter': [{'range': {'path': 'year', 'gte': 2013, 'lte': 2015}}],
        'should': [{'text': {'query': 'snow', 'path': 'title', 'score': {'constant': {'value': 5}}}}]},
      'highlight': {'path': 'title'}}},
  {'$limit': 10}, 
  {'$project': {'_id': 0, 'title': 1, 'year': 1, 
    'score': {'$meta': 'searchScore'}, "highlights": {"$meta": "searchHighlights"}}}
]

result = client['sample_mflix']['movies'].aggregate(pipeline)
# print results
for i in result:
    print(i)

{'title': 'Snow in Paradise', 'year': 2014, 'score': 5.0, 'highlights': [{'score': 1.382846713066101, 'path': 'title', 'texts': [{'value': 'Snow', 'type': 'hit'}, {'value': ' in Paradise', 'type': 'text'}]}]}
{'title': 'Dead Snow 2: Red vs. Dead', 'year': 2014, 'score': 5.0, 'highlights': [{'score': 1.3924485445022583, 'path': 'title', 'texts': [{'value': 'Dead ', 'type': 'text'}, {'value': 'Snow', 'type': 'hit'}, {'value': ' 2: Red vs. ', 'type': 'text'}]}]}
{'title': 'The Snow White Murder Case', 'year': 2014, 'score': 5.0, 'highlights': [{'score': 1.3525336980819702, 'path': 'title', 'texts': [{'value': 'The ', 'type': 'text'}, {'value': 'Snow', 'type': 'hit'}, {'value': ' White Murder Case', 'type': 'text'}]}]}
{'title': 'Snow on the Blades', 'year': 2014, 'score': 5.0, 'highlights': [{'score': 1.3766303062438965, 'path': 'title', 'texts': [{'value': 'Snow', 'type': 'hit'}, {'value': ' on the Blades', 'type': 'text'}]}]}
{'year': 2013, 'title': 'The Secret Life of Walter Mitty', 's

In [8]:
# 2013 년부터 2015년에 year 필드 레인지에서 
# title 에 snow 가 들어가는 document 를  찾아라 
# 매칭이 되면 Score 가 변하는 게 중요 해당 Score 가 boost 인것을 주목 


pipeline = [
  {'$search': {
      'compound': {
        'must': [{'range': {'path': 'year', 'gte': 2013, 'lte': 2015}}],
        'should': [{'text': {'query': 'snow', 'path': 'title', 'score': {'boost': {'value': 2}}}}]}, 
      'highlight': {'path': 'title'}}},
  {'$limit': 10}, 
  {'$project': {'_id': 0, 'title': 1, 'year': 1, 'score': {'$meta': 'searchScore'}, "highlights": {"$meta": "searchHighlights"}}}
]
# run pipeline
result = client['sample_mflix']['movies'].aggregate(pipeline)
# print results
for i in result:
    print(i)

{'title': 'Snow in Paradise', 'year': 2014, 'score': 6.7722930908203125, 'highlights': [{'score': 1.382846713066101, 'path': 'title', 'texts': [{'value': 'Snow', 'type': 'hit'}, {'value': ' in Paradise', 'type': 'text'}]}]}
{'title': 'Snow on the Blades', 'year': 2014, 'score': 6.063445568084717, 'highlights': [{'score': 1.3766303062438965, 'path': 'title', 'texts': [{'value': 'Snow', 'type': 'hit'}, {'value': ' on the Blades', 'type': 'text'}]}]}
{'title': 'The Snow White Murder Case', 'year': 2014, 'score': 5.509652137756348, 'highlights': [{'score': 1.3525336980819702, 'path': 'title', 'texts': [{'value': 'The ', 'type': 'text'}, {'value': 'Snow', 'type': 'hit'}, {'value': ' White Murder Case', 'type': 'text'}]}]}
{'title': 'Dead Snow 2: Red vs. Dead', 'year': 2014, 'score': 5.065053939819336, 'highlights': [{'score': 1.3924485445022583, 'path': 'title', 'texts': [{'value': 'Dead ', 'type': 'text'}, {'value': 'Snow', 'type': 'hit'}, {'value': ' 2: Red vs. ', 'type': 'text'}]}]}
{'ye

In [10]:
## 각각의 필드별로 가중치를 다르게 준 예제 

result = client['sample_mflix']['movies'].aggregate([
    {
        '$search': {
            'index': 'default',
            'compound': {
                'must': [
                    {
                        'text': {
                            'path': 'genres',
                            'query': 'comedy',
                            'score': {
                                'boost': {
                                    'value': 9
                                }
                            }
                        }
                    }, {
                        'text': {
                            'path': 'title',
                            'query': 'snow',
                            'score': {
                                'boost': {
                                    'value': 5
                                }
                            }
                        }
                    }
                ],
                'should': [
                    {
                        'range': {
                            'path': 'year',
                            'gte': 2013,
                            'lte': 2015,
                            'score': {
                                'boost': {
                                    'value': 3
                                }
                            }
                        }
                    }
                ]
            }
        }
    }, {
        '$limit': 10
    }, {
        '$project': {
            '_id': 0,
            'title': 1,
            'year': 1,
            'genres': 1,
            'score': {
                '$meta': 'searchScore'
            }
        }
    }
])
for i in result:
    print(i)

{'genres': ['Comedy', 'Horror'], 'title': 'Dead Snow', 'year': 2009, 'score': 21.872983932495117}
{'year': 2000, 'genres': ['Adventure', 'Comedy', 'Family'], 'title': 'Snow Day', 'score': 21.043487548828125}
{'genres': ['Adventure', 'Comedy', 'Family'], 'title': 'Snow Dogs', 'year': 2002, 'score': 21.043487548828125}
{'year': 1999, 'genres': ['Comedy', 'Romance'], 'title': 'Let It Snow', 'score': 19.523927688598633}
{'genres': ['Action', 'Comedy', 'Horror'], 'title': 'Dead Snow 2: Red vs. Dead', 'year': 2014, 'score': 17.426334381103516}
{'genres': ['Comedy', 'Drama'], 'title': 'Snow White and Russian Red', 'year': 2009, 'score': 16.367326736450195}
{'genres': ['Comedy', 'Drama', 'Romance'], 'title': 'The Tiger and the Snow', 'year': 2005, 'score': 15.537829399108887}
{'genres': ['Adventure', 'Comedy', 'Family'], 'title': 'Snow White and the Three Stooges', 'year': 1961, 'score': 14.4263334274292}


In [4]:
## 이건 사용자 함수를 등록해서 처리한 사례 

pipeline = [
  {'$search': {
      'compound': {
        'must': [{'range': {'path': 'year', 'gte': 2013, 'lte': 2015}}],
        'should': [{'text': {'query': 'snow', 'path': 'title', 
                    'score': {'function': {
                        'add': [{'path': {'value': 'imdb.rating','undefined': 2}}, {'score': 'relevance'}]}}}}]},
      'highlight': {'path': 'title'}}},
  {'$limit': 10}, 
  {'$project': {'_id': 0, 'title': 1, 'year': 1, 'score': {'$meta': 'searchScore'}, "highlights": {"$meta": "searchHighlights"}}}
]
# run pipeline
result = client['sample_mflix']['movies'].aggregate(pipeline)
# print results
for i in result:
    print(i)


{'title': 'The Snow White Murder Case', 'year': 2014, 'score': 10.454826354980469, 'highlights': [{'score': 1.3525336980819702, 'path': 'title', 'texts': [{'value': 'The ', 'type': 'text'}, {'value': 'Snow', 'type': 'hit'}, {'value': ' White Murder Case', 'type': 'text'}]}]}
{'title': 'Snow on the Blades', 'year': 2014, 'score': 10.3317232131958, 'highlights': [{'score': 1.3766303062438965, 'path': 'title', 'texts': [{'value': 'Snow', 'type': 'hit'}, {'value': ' on the Blades', 'type': 'text'}]}]}
{'title': 'Dead Snow 2: Red vs. Dead', 'year': 2014, 'score': 10.032526969909668, 'highlights': [{'score': 1.3924485445022583, 'path': 'title', 'texts': [{'value': 'Dead ', 'type': 'text'}, {'value': 'Snow', 'type': 'hit'}, {'value': ' 2: Red vs. ', 'type': 'text'}]}]}
{'title': 'Snow in Paradise', 'year': 2014, 'score': 8.386146545410156, 'highlights': [{'score': 1.382846713066101, 'path': 'title', 'texts': [{'value': 'Snow', 'type': 'hit'}, {'value': ' in Paradise', 'type': 'text'}]}]}
{'ye